In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import os
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as torchdata
import fastai

# Fast AI Time Series Competition No1: Earthquakes.

Data is already downloaded in `../data/`.

## Data Loading

In [2]:
data_path = pathlib.Path('../data')
batch_size = 128

In [3]:
os.listdir(data_path)

['Earthquakes_TEST.txt',
 'Earthquakes.txt',
 'Earthquakes_TRAIN.arff',
 'GADF',
 'Earthquakes_TRAIN.txt',
 'Earthquakes_TEST.arff']

In [4]:
train = np.loadtxt(data_path/'Earthquakes_TRAIN.txt')
test = np.loadtxt(data_path/'Earthquakes_TEST.txt')

In [5]:
train, train_labels = train[:, 1:], train[:, 0].astype(np.int)
test, test_labels = test[:, 1:], test[:, 0].astype(np.int)

In [6]:
train_ds = torchdata.TensorDataset(
    torch.tensor(train, dtype=torch.float32),
    torch.tensor(train_labels, dtype=torch.float32).unsqueeze(1))
test_ds = torchdata.TensorDataset(
    torch.tensor(test, dtype=torch.float32),
    torch.tensor(test_labels, dtype=torch.float32).unsqueeze(1))

In [7]:
train_dl = torchdata.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = torchdata.DataLoader(test_ds, batch_size=batch_size, shuffle=True)

In [8]:
data = fastai.DataBunch(train_dl, test_dl)

## LSTM-FCNN
This is a reimplementation of the LSTM-FCNN from https://ieeexplore-ieee-org.ezproxy.cul.columbia.edu/ielx7/6287639/8274985/08141873.pdf?tp=&arnumber=8141873&isnumber=8274985&tag=1

The reported accuracy on the Earthquakes dataset in the paper is .8354.

In [9]:
class ShuffleLSTM(nn.Module):
    def __init__(self, seq_length, dropout=0.0, hidden_size=128, num_layers=1, bidirectional=False):
        super().__init__()
        self.lstm = nn.LSTM(input_size=seq_length,
                           hidden_size=hidden_size,
                           num_layers=num_layers,
                           bidirectional=bidirectional)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        ## input size will be the batch, input_size, seq_len =  64, 1, 512
        ## we start with the shuffle layer, transforming this in a 1 step multivariate TS:
        ## output should be 1, 64, 512
        x = torch.transpose(x, 0, 1)
        # input_size, batch, seq_len
        out, (_, _) = self.lstm(x)
        # after the lstm, the output will be 1, bs, lstm_size
        return self.dropout(out)

In [10]:
class ConvLayer(nn.Module):
    def __init__(self, in_channels=1, n_channels=128, kernel_size=8, dropout=0.0):
        super().__init__()
        self.conv = nn.Conv1d(in_channels=in_channels,
                             out_channels=n_channels,
                             kernel_size=kernel_size)
        self.bn = nn.BatchNorm1d(num_features=n_channels)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # input size should be batch, input_size, length = 64, 1, 312 at first
        x = self.conv(x)
        # now batch, n_channels, L
        x = self.bn(x)
        x = F.relu(x)
        # output will be bs, n_filters, L (not equal to inital L)
        return self.dropout(x)
    

In [11]:
class LSTM_FCNN(nn.Module):
    def __init__(self,
                 seq_length,
                 lstm_dropout=0.0,
                 lstm_hidden_size=128,
                 lstm_num_layers=1,
                 lstm_bidirectional=False,
                 conv_n_channels = [128, 256, 128],
                 conv_kernel_sizes = [8, 5, 3],
                 conv_dropout=0.0):
        
        super().__init__()
        
        self.lstm = ShuffleLSTM(seq_length,
                                lstm_dropout,
                                lstm_hidden_size,
                                lstm_num_layers,
                                lstm_bidirectional)
        self.conv1 = ConvLayer(1, conv_n_channels[0], conv_kernel_sizes[0])  
        self.conv2 = ConvLayer(conv_n_channels[0], conv_n_channels[1], conv_kernel_sizes[1])  
        self.conv3 = ConvLayer(conv_n_channels[1], conv_n_channels[2], conv_kernel_sizes[2])  
        
        # global pool is avg pool using the length of the resulting TS
        # we need to calculate this:
        out_len = seq_length - sum(conv_kernel_sizes) + len(conv_kernel_sizes)
        self.global_avg_pool = nn.AvgPool1d(kernel_size = out_len)
        
        
        self.linear = nn.Linear(conv_n_channels[-1] + lstm_hidden_size, 1)
        
        
        
    def forward(self, x):
        # our input is batch, seq_len
        x = x.unsqueeze(1)
        # batch, 1, seq_len
        
        
        lstm_out = self.lstm(x)
        lstm_out = torch.squeeze(lstm_out)
        # lstm_out is bs, lstm_size
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        # x is now bs, dimension, L
        # avgpool1d averages over the last dimension so we transpose again
        x = self.global_avg_pool(x)
        # bs, dimension, 1
        x = torch.squeeze(x)
        # bs, dimension
        
        concat = torch.cat((lstm_out, x), 1)
        
        return self.linear(concat)


## Training

In [20]:
model = LSTM_FCNN(512, lstm_dropout=0.8, lstm_hidden_size=8)

In [21]:
pos_weight = len(train_labels)/ np.sum(train_labels) -1
pos_weight

4.551724137931035

In [22]:
#loss = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight], dtype=torch.float32))
loss = nn.BCEWithLogitsLoss()
loss = loss.cuda()

In [23]:
def my_acc(pred, targ):
    ok = torch.tensor(torch.abs(torch.sign(pred)/2 + .5 - targ) < .01, dtype=torch.float32)
    return torch.mean(ok)

In [24]:
learner = fastai.Learner(data, model, loss_func=loss, metrics=my_acc)
learner.unfreeze()

In [25]:
#learner.lr_find(); learner.recorder.plot()

In [26]:
learner.fit(200, lr=1e-3)

Total time: 00:28
epoch  train_loss  valid_loss  my_acc  
1      0.568187    0.672679    0.719424  (00:00)
2      0.548042    0.666089    0.726619  (00:00)
3      0.531417    0.649852    0.733813  (00:00)
4      0.521316    0.627779    0.741007  (00:00)
5      0.513305    0.600121    0.748201  (00:00)
6      0.505123    0.575806    0.748201  (00:00)
7      0.496105    0.559037    0.748201  (00:00)
8      0.489058    0.546101    0.748201  (00:00)
9      0.481128    0.535779    0.748201  (00:00)
10     0.474845    0.528560    0.748201  (00:00)
11     0.470002    0.525075    0.748201  (00:00)
12     0.466731    0.522482    0.748201  (00:00)
13     0.462355    0.520715    0.748201  (00:00)
14     0.458115    0.518622    0.748201  (00:00)
15     0.456863    0.518696    0.748201  (00:00)
16     0.452798    0.518607    0.748201  (00:00)
17     0.450655    0.519165    0.748201  (00:00)
18     0.446936    0.576442    0.748201  (00:00)
19     0.443479    0.530901    0.748201  (00:00)
20     0.44